<a href="https://colab.research.google.com/github/soaeng/recomSys/blob/main/CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 **기본 CF 알고리즘**

## 🔷 **데이터 불러오기**

In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

base_src = 'drive/MyDrive/RecoSys/Data'

# user 데이터
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src, sep = '|', names = u_cols, encoding = 'latin-1')
users = users.set_index('user_id')

# movie 데이터
u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id', 'title', 'release data', 'video release data', 'IMDB URL',
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv(u_item_src, sep = '|', names = i_cols, encoding = 'latin-1')
movies = movies.set_index('movie_id')

# rating 데이터
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep = '\t', names = r_cols, encoding = 'latin-1')

## 🔷 **정확도 예측 함수**

In [17]:
# 정확도(RMSE) 계산
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# 모델별 RMSE 계산
def score(model):
  # 테스트 데이터의 user_id와 movie_id 간 pair를 맞춰 튜플형 원소 리스트 데이터를 생성
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  # 모든 사용자-영화 pair에 대해서 주어진 예측 모델에 의한 예측값 계산 및 리스트 데이터 생성
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  # 실제 평점값
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

# 데이터셋 생성
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)
rating_matrix = x_train.pivot(index = 'user_id', columns = 'movie_id', values = 'rating')

## 🔷 **코사인 유사도 계산**

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.131524,0.029309,0.029267,0.270231,0.337902,0.317028,0.216077,0.065790,0.260959,...,0.267283,0.118663,0.252868,0.120509,0.180525,0.113500,0.238995,0.130502,0.113670,0.281856
2,0.131524,1.000000,0.098981,0.117372,0.034053,0.159846,0.071211,0.058660,0.081954,0.089882,...,0.089813,0.175860,0.259162,0.179916,0.234674,0.152453,0.214636,0.072848,0.107027,0.086360
3,0.029309,0.098981,1.000000,0.273115,0.029405,0.078473,0.054825,0.060784,0.062904,0.055192,...,0.019276,0.039268,0.139763,0.077143,0.060166,0.020060,0.137381,0.088066,0.068247,0.037120
4,0.029267,0.117372,0.273115,1.000000,0.013213,0.040300,0.046498,0.111279,0.000000,0.037096,...,0.000000,0.000000,0.110322,0.085593,0.098465,0.000000,0.148897,0.075378,0.110338,0.037068
5,0.270231,0.034053,0.029405,0.013213,1.000000,0.179919,0.299594,0.190298,0.040578,0.160485,...,0.228165,0.058260,0.052892,0.060452,0.126783,0.079711,0.179834,0.086024,0.099788,0.185404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.113500,0.152453,0.020060,0.000000,0.079711,0.101034,0.119510,0.064197,0.000000,0.064270,...,0.046654,0.354247,0.150892,0.154904,0.409508,1.000000,0.117964,0.236958,0.035468,0.095212
940,0.238995,0.214636,0.137381,0.148897,0.179834,0.260398,0.246882,0.206126,0.112824,0.275252,...,0.247956,0.062059,0.163595,0.122653,0.172973,0.117964,1.000000,0.127620,0.189200,0.111484
941,0.130502,0.072848,0.088066,0.075378,0.086024,0.131185,0.036481,0.100655,0.000000,0.057415,...,0.061179,0.184239,0.207195,0.145094,0.328416,0.236958,0.127620,1.000000,0.100457,0.080496


## 🔷 **가중평균 rating 계산**

In [18]:
def CF_simple(user_id, movie_id):
  if movie_id in rating_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  else:
    mean_rating = 3.0
  return mean_rating

score(CF_simple)

1.0158446410086883

---
# 📌 **이웃을 고려한 CF**

In [22]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

base_src = 'drive/MyDrive/RecoSys/Data'

# user 데이터
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src, sep = '|', names = u_cols, encoding = 'latin-1')
users = users.set_index('user_id')

# movie 데이터
u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id', 'title', 'release data', 'video release data', 'IMDB URL',
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv(u_item_src, sep = '|', names = i_cols, encoding = 'latin-1')
movies = movies.set_index('movie_id')

# rating 데이터
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep = '\t', names = r_cols, encoding = 'latin-1')

# 정확도(RMSE) 계산
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# 모델별 RMSE 계산
def score(model, neighbor_size = 0):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

# 데이터셋 생성
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)
rating_matrix = x_train.pivot(index = 'user_id', columns = 'movie_id', values = 'rating')

# 코사인 유사도 계산
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

def CF_knn(user_id, movie_id, neighbor_size = 0):
  # train/test set의 분할에 따라 rating_matrix에 해당 영화가 있는지 확인
  if movie_id in rating_matrix.columns:
    # 주어진 사용자(user_id)와 다른 사용자의 유사도 추출
    sim_scores = user_similarity[user_id].copy()
    # 주어진 영화(movie_id)와 다른 사용자의 유사도 추출
    movie_ratings = rating_matrix[movie_id].copy()
    # 주어진 영화에 대해 평가하지 않은 사용자를 가중평균계산에서 제외하기 위해 인덱스 추출
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    # 위에서 뽑은 사람 제외
    movie_ratings = movie_ratings.dropna()
    # 평가하지 않은 사용자와의 유사도 제거
    sim_scores = sim_scores.drop(none_rating_idx)

    # neighbor_size가 지정되지 않은 경우
    if neighbor_size == 0:
      mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

    # neighbor_size가 지정된 경우
    else :
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      else:
        mean_rating = 3.0
  else:
    mean_rating = 3.0
  return mean_rating

score(CF_knn, neighbor_size = 30)

# neighbor size에 변화하는 RMSE 계산
for neighbor_size in [10, 20, 30, 40, 50, 60]:
  print('Neighbor size = %d: RMSE = %.4f' %(neighbor_size, score(CF_knn, neighbor_size)))

Neighbor size = 10: RMSE = 1.0317
Neighbor size = 20: RMSE = 1.0168
Neighbor size = 30: RMSE = 1.0138
Neighbor size = 40: RMSE = 1.0139
Neighbor size = 50: RMSE = 1.0140
Neighbor size = 60: RMSE = 1.0143


## 🔷 **실제 주어진 사용자에 대해 추천을 받는 기능**

In [23]:
def recom_movie(user_id, n_items, neighbor_size = 30):
  user_movie = rating_matrix.loc[user_id].copy()

  for movie in rating_matrix.columns:
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0
    else:
      user_movie.loc[movie] = CF_knn(user_id, movie, neighbor_size)

  movie_sort = user_movie.sort_values(ascending = False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']
  return recommendations

recom_movie(user_id = 729, n_items = 5, neighbor_size = 30)

movie_id
1189           Prefontaine (1997)
1516     Wedding Gift, The (1994)
1500    Santa with Muscles (1996)
1293              Star Kid (1997)
1612      Leading Man, The (1996)
Name: title, dtype: object